<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**검증 세트**

하이퍼파라미터를 튜닝하면서 테스트세트로 결과를 계속 확인하면 테스트 세트를 훈련세트처럼 학습하게되어 신뢰도가 떨어짐

테스트 세트를 마지막에만 사용하기 위해 훈련세트에서 검증세트를 다시 분리

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

훈련세트에서 검증세트 분리

In [5]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [6]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [9]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


**교차검증**

많은 데이터를 훈련에 사용할수록 좋은 모델이 형성되지만 검증세트를 적게 분리하면 검증 데이터 점수 신뢰도가 떨어짐

교차검증을 통해 안정적인 검증 점수를 얻고, 훈련에 많은 데이터 활용

검증세트 분리하고 평가하는 과정을 여러번 반복해 점수를 평균하여 최종점수를 얻음

통상 5 or 10 fold 교차검증 사용

In [10]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target) # 기본값 5
print(scores) # test_score -> 검증 폴드 점수

{'fit_time': array([0.01327014, 0.0311451 , 0.03016615, 0.01050544, 0.01891351]), 'score_time': array([0.01198196, 0.00193143, 0.0018239 , 0.00175261, 0.00174904]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [11]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [13]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold()) # 분류 -> StratifiedKFold() / 회귀 -> KFold()
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


**하이퍼파라미터 튜닝**

* AutoML = 사람 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행

트리모델에서 최적 max_depth를 찾아서 고정시키고 다른 매개변수를 바꾸면 최적값이 바뀜

그리드서치 -> GridSearchCV

- 하이퍼파라미터 탐색, 교차검증을 동시에 수행(cross_validate 수행 불필요)

In [16]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = 1)
# 'min_inpurity_decrease' 값마다 5-폴드 교차검증 수행 -> 5 * 5 수행
# n_jobs -> 병렬 실행 CPU 코어 수 지정 / 기본값 1 / 모든코어 -1

gs.fit(train_input, train_target)

dt = gs.best_estimator_ # 검증 점수가 가장 높은 모델
print(dt.score(train_input, train_target))

0.9615162593804117


In [17]:
print(gs.best_params_) # 최적 매개변수

{'min_impurity_decrease': 0.0001}


In [19]:
print(gs.cv_results_['mean_test_score']) # 각 매개변수에서 수행한 교차 검증 평균 점수

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [21]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # 가장 큰 값의 인덱스 추출
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [22]:
params = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001), # 0.0001 에서 시작해 0.0001씩 더해 0.001이 될 때까지 더한 배열 / 9개
          'max_depth':range(5,20,1), # 15개  # range -> 정수만 사용 가능
          'min_samples_split': range(2,100,10)} # 50개
          # 검증횟수 -> 9 * 15 * 50 = 1350
          # 모델수 -> 5(k폴드 교차 검증) * 9 * 15 * 50 = 6750

In [23]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [24]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


**랜덤서치**

매개변수 값의 목록을 전달하지 않고 매개변수를 샘플링할 수 있는 확률 분포객체를 전달

연속된 매개변수 값을 탐색할 때 유용

싸이파이 라이브러리

파이썬 과학 라이브러리

적분, 보간, 선형대수, 확률 등을 포함한 수치 계산 전용 라이브러리

In [26]:
from scipy.stats import uniform, randint

rgen = randint(0, 10) # random_int
rgen.rvs(10) # 인덱스 별 추출된 수를 출력

array([1, 6, 8, 9, 2, 3, 7, 6, 6, 8])

균등분포에서 샘플링 = 주어진 범위에서 고르게 값을 추출

In [28]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 92,  95, 102,  96, 117, 101, 110,  97, 111,  79]))

In [29]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.62632906, 0.90485847, 0.31943402, 0.86400082, 0.61274943,
       0.88031667, 0.5396539 , 0.1096492 , 0.88782978, 0.17644325])

In [30]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1, 25)}

In [31]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = 1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c8b7132ca60>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c8b7132ff10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c8b7132f4c0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c8b7132df90>},
                   random_state=42)

In [32]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [34]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [35]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
